$\newcommand{\xv}{\mathbf{x}}
\newcommand{\Xv}{\mathbf{X}}
\newcommand{\yv}{\mathbf{y}}
\newcommand{\zv}{\mathbf{z}}
\newcommand{\av}{\mathbf{a}}
\newcommand{\Wv}{\mathbf{W}}
\newcommand{\wv}{\mathbf{w}}
\newcommand{\tv}{\mathbf{t}}
\newcommand{\Tv}{\mathbf{T}}
\newcommand{\muv}{\boldsymbol{\mu}}
\newcommand{\sigmav}{\boldsymbol{\sigma}}
\newcommand{\phiv}{\boldsymbol{\phi}}
\newcommand{\Phiv}{\boldsymbol{\Phi}}
\newcommand{\Sigmav}{\boldsymbol{\Sigma}}
\newcommand{\Lambdav}{\boldsymbol{\Lambda}}
\newcommand{\half}{\frac{1}{2}}
\newcommand{\argmax}[1]{\underset{#1}{\operatorname{argmax}}}
\newcommand{\argmin}[1]{\underset{#1}{\operatorname{argmin}}}$

# Assignment 3: Neural Network Regression

*Sean Russell*

Sorry, got my time management all wrong. Thus all I have for you is code.

## Python Implementations

In [1]:
import neuralnetworks as nn
import numpy as np

In [2]:
def trainLinear(X,T,lamb):
    means = X.mean(0)
    stds = X.std(0)
    n,d = X.shape
    Xs1 = np.insert( (X - means)/stds, 0, 1, axis=1)
    lambDiag = np.eye(d+1) * lamb
    lambDiag[0,0] = 0
    w = np.linalg.lstsq( Xs1.T @ Xs1 + lambDiag, Xs1.T @ T)[0]
    return {'w': w, 'means':means, 'stds':stds}

def evaluateLinear(model,X,T):
    Xs1 = np.insert((X-model['means'])/model['stds'], 0, 1, axis=1)
    evalT = Xs1 @ model['w']
    return np.sqrt(np.mean( (evalT-T)**2 ))

def trainNN(X,T,parameters):
    # parameters is list of number of hidden layers and nodes and number of iterations to train
    # looks like [hidden layer, number of iterations]
    # where hidden layer could be integer or list
    nnet = nn.NeuralNetwork(1, parameters[0], 1)  # 1 input, 5 hidden units, 1 output
    nnet.train(X, T, nIterations=parameters[1])
    return nnet

def evaluateNN(model,X,T):
    predict = model.use(X)
    return np.sqrt(np.mean((predict - T)**2))

In [3]:
def trainValidateTestKFolds(trainf,evaluatef,X,T,parameterSets,nFolds,
                            shuffle=False,verbose=False):
    # Randomly arrange row indices
    rowIndices = np.arange(X.shape[0])
    if shuffle:
        np.random.shuffle(rowIndices)
    # Calculate number of samples in each of the nFolds folds
    nSamples = X.shape[0]
    nEach = int(nSamples / nFolds)
    if nEach == 0:
        raise ValueError("partitionKFolds: Number of samples in each fold is 0.")
    # Calculate the starting and stopping row index for each fold.
    # Store in startsStops as list of (start,stop) pairs
    starts = np.arange(0,nEach*nFolds,nEach)
    stops = starts + nEach
    stops[-1] = nSamples
    startsStops = list(zip(starts,stops))
    # Repeat with testFold taking each single fold, one at a time
    results = []

    # COMPLETE THIS FUNCTION BY IMPLEMENTING THE FOLLOWING STEPS.

    # For each test fold
    for n in range(nFolds):
        testX = X[slice(*startsStops[n]),:]
        testT = T[slice(*startsStops[n]),:]
        valError = float("inf")
        bestParm = parameterSets[0]
        for parmSet in parameterSets:
            errors = []
            for i in range(nFolds):
                if i == n:
                    continue
                
                valX = X[slice(*startsStops[i]),:]
                valT = T[slice(*startsStops[i]),:]
                trainX = np.delete(np.delete(X,slice(*startsStops[i]),0),slice(*startsStops[n]),0)
                trainT = np.delete(np.delete(T,slice(*startsStops[i]),0),slice(*startsStops[n]),0)
                if (n > i):
                    trainX = np.delete(np.delete(X,slice(*startsStops[n]),0),slice(*startsStops[i]),0)
                    trainT = np.delete(np.delete(T,slice(*startsStops[n]),0),slice(*startsStops[i]),0)
                model = trainf(trainX,trainT,parmSet)
                errors.append(evaluatef(model,valX,valT))
            
            meanError = sum(errors)/len(errors)
            if meanError < valError:
                valError = meanError
                bestParm = parmSet
        
        trainX = np.delete(X,slice(*startsStops[n]),0)
        trainT = np.delete(T,slice(*startsStops[n]),0)
        testModel = trainf(trainX,trainT,bestParm)
        trainError = evaluatef(testModel,trainX,trainT)
        testError = evaluatef(testModel,testX,testT)
        testResults = [bestParm,trainError,valError,testError]
        results.append(testResults)
        if verbose:
            print (testResults)
    
    return results

In [4]:
%run -i "A3grader.py"

 Testing: result = trainValidateTestKFolds(trainLinear,evaluateLinear,X,T,
                  range(0,101,10),nFolds=5,shuffle=False)
 Your result is
    10   3.158   4.132   2.414
    20   4.368   5.021   3.641
    10   3.245   4.178   5.03
    20   4.448   6.07   2.024
    20   2.426   2.972   10.89
20/20 points. First column, of best lambda values, is correct.
20/20 points. Columns of RMSE values are correct.

 Testing:
   import itertools
   parms = list(itertools.product([[5],[5,5],[2,2,2]], [10,50,100,200]))
   te = []
   for rep in range(5):
       result = trainValidateTestKFolds(trainNN,evaluateNN,X,T,
                                        parms,
                                        nFolds=4,shuffle=False)
       resulte = np.array([r[1:] for r in result])
       meanTestRMSE = resulte[:,-1].mean()
       print('     ',meanTestRMSE)
       te.append(meanTestRMSE)
      1.68887330008
      1.54186238162
      1.27425834675
      1.6353607673
      1.79208214933
40/40 points